In [ ]:
#import pandas as pd
import numpy as np
import plotly.express as px
import experiment_helpers

#debug
from importlib import reload
from experiment_helpers import *
reload(experiment_helpers)
from experiment_helpers import *

In [ ]:
EXPERIMENT_TYPES = [
    "no_prefetching",
    "linux_prefetching",
    #"linux_prefetching_ssdopt",
    #"linux_prefetching_ssdopt_asyncwrites","linux_prefetching_ssdopt_asyncwrites",
    #"tape_prefetching_asyncwrites","tape_prefetching_syncwrites"
]
annotations=[]

In [ ]:
def add_annotation(text):
    anno = "<h3><pre>%s</pre></h3>" % text
    annotations.append(anno)

# Experiment plots

## Define experiments

In [ ]:
WORKLOADS = ["mmult_eigen","vec_eigen",
             #"dot_eigen", "mmap_random_rw","kissfft",  "kmeans", "torch",
             "bitonic_merge", "native_sort",
             "bitonic_sort", "torch", "torch_par","torch_par4","linpack", "mmult_eigen_par"]
#"alexnet", "vgg16", "batcher_sort"]:

## Load data:       data[EXPERIMENT_TYPE][WORKLOAD]

In [ ]:
from collections import defaultdict

data = defaultdict(dict)
for experiment_type in EXPERIMENT_TYPES:
    for workload in WORKLOADS:
        data[experiment_type][workload] = augment_tables(get_experiment_data([experiment_type], workload,"../experiment_results_4us"), filter_raw=True)[0]


## Explore & Plot

In [ ]:
plots = []

for experiment_type in EXPERIMENT_TYPES:
    named_workloads = list(zip(WORKLOADS, data[experiment_type].values()))
    
    runtime = take_column_named("Measured(wallclock) runtime",named_workloads).dropna()
    fig = px.line(runtime, title='(%s)Absolute(Wallclock) Runtime vs. Local Memory'% experiment_type)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Measured(wallclock) runtime(s)",
    )
    fig.show()
    plots.append(fig)
    with open('figs/absolute.html', 'w') as f:
        f.write(fig.to_html(full_html=False, include_plotlyjs='plotly-latest.min.js'))
    
    
    degradations = take_column_named("Degradation",named_workloads).dropna() * 100
    fig = px.line(degradations, title='(%s)Normalized Runtime(Wallclock) vs. Local Memory' % experiment_type)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Degradation(%)",
    )
    fig.show()
    plots.append(fig)
    with open('figs/degr.html', 'w') as f:
        f.write(fig.to_html(full_html=False, include_plotlyjs='plotly-latest.min.js'))

    degradations_no_ev = take_column_named("Degradation w/o Evictions",named_workloads).dropna() * 100
    fig = px.line(degradations_no_ev, title='(%s)Normalized Runtime vs. Local Memory (SUBTRACT time spent in evictions)' % experiment_type)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Degradation(%)",
    )
    fig.show()
    plots.append(fig)
    
    with open('figs/degr_noev.html', 'w') as f:
        f.write(fig.to_html(full_html=False, include_plotlyjs='plotly-latest.min.js'))

In [ ]:
runtime_components = []

for workload in WORKLOADS:
    for exp_type in EXPERIMENT_TYPES:
        fig = get_components_of_runtime(data[exp_type][workload], "%s,%s" % (exp_type, workload))
        runtime_components.append(fig)
    #    fig.update_layout(yaxis_range=[0,15])
        with open('figs/comp_%s_%s.html' % (exp_type, workload), 'w') as f:
            f.write(fig.to_html(full_html=False, include_plotlyjs='plotly-latest.min.js'))

        fig.show()

In [ ]:
(exp_type, workload) = "linux_prefetching", "torch_par"
get_components_of_runtime(data[exp_type][workload], "%s,%s, +4us rdma delay" % (exp_type, workload))

In [ ]:
data['linux_prefetching']['bitonic_sort']['Minor Page Faults']/ \
data['linux_prefetching']['bitonic_sort']['Major Page Faults']

In [ ]:
for exp_name in ["tape_prefetching_syncwrites", "tape_prefetching_asyncwrites"]:
    fig = get_components_of_runtime(get_experiment_table(named_dfs,exp_name))

    runtime_components.append(fig)
    fig.update_layout(yaxis_range=[0,15])
    fig.show()


In [ ]:
def spill_html():
    with open('linux_delay_8.html', 'w') as f:
        i = 0
        for figures in [plots,runtime_components]:
            for figure in figures:
                #anno = annotations[i]
                #i+=1
                f.write(figure.to_html(full_html=False, include_plotlyjs='cdn'))
                #f.write(anno)
                f.write("<hr/>")